In [1]:
### JUPYTER NOTEBOOK / MODULE PATH FIX ###
import sys

if "/share/apps/python/3.8.6/intel/lib/python3.8" in sys.path and sys.path[1] == '':
    sys.path[1], sys.path[6] = sys.path[6], sys.path[1]

In [2]:
from jtb_2022_code.utils.figure_common import *
from jtb_2022_code.utils.adata_common import *
from jtb_2022_code.utils.pseudotime_common import (do_pca_pt, spearman_rho_pools, spearman_rho_grid, 
                                                   do_denoised_pca, calc_rhos, do_time_assign_by_pool)
from jtb_2022_code import FigureSingleCellData

from inferelator_prior.velocity.programs import sparse_PCA

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.stats import spearmanr

In [3]:
data = FigureSingleCellData()
data.apply_inplace_to_expts(sparse_PCA, alphas=np.array([0.1, 1, 10]))

Reading Single Cell Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_1_WT.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_1_fpr1.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_2_WT.h5ad
Reading Single Cell Experiment Data from /scratch/cj59/RAPA/2021_RAPA_TIMECOURSE_FIGS_2_fpr1.h5ad


100%|██████████| 3/3 [01:15<00:00, 25.24s/it]


In [5]:
data

In [8]:
data.expt_data[(2, "WT")].uns['sparse_pca']

{'alphas': array([ 0.1,  1. , 10. ]),
 'loadings': [array([[-5.24493756e-05, -2.06855383e-03,  1.93848933e-03, ...,
           9.20310946e-03, -2.51396760e-04,  1.01339261e-02],
         [-4.48367307e-03, -8.05384433e-03, -9.42495119e-03, ...,
           1.28339190e-02,  3.04614743e-02, -1.67541379e-02],
         [-6.43130533e-04,  1.43188058e-03,  3.34804820e-04, ...,
          -3.28065231e-03,  1.17416526e-03, -9.85077643e-03],
         ...,
         [-9.47236155e-05,  3.21590581e-03,  6.20944084e-03, ...,
           1.21783145e-02,  1.79180435e-02,  8.68706707e-03],
         [-5.79735906e-04, -1.14150401e-03, -2.85502010e-03, ...,
           0.00000000e+00, -1.48315903e-02,  1.53080187e-02],
         [ 3.41276288e-05,  2.25688225e-03,  5.32980494e-03, ...,
          -2.76652205e-03,  1.61084729e-02,  2.88053316e-03]]),
  array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           7.01259576e-03,  0.00000000e+00,  0.00000000e+00],
         [-2.72119010e-03, -7.18549264